In [1]:
import os 
from dotenv import load_dotenv
load_dotenv('/media/varun/Data/data science/main_files/.env')

True

In [2]:
import os 
import google.generativeai as genai 
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate

#set the environment variables
os.environ['GOOGLE_API_KEY']=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

#load the model 
model=genai.GenerativeModel('gemini-pro')
llm = ChatGoogleGenerativeAI(model="gemini-pro")

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from langchain_groq import ChatGroq
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')

#load the model
llm = ChatGroq(model='gemma2-9b-it')

system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm

zero shot agent - have no memory only interact once 



In [4]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [5]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [6]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [7]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [8]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [9]:
for obs in observations:
    insert_obs(obs)

In [10]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_experimental/sql/base.py:77: UserWarning: Directly instantiating an SQLDatabaseChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [11]:
os.environ['OPENAI_API_KEY'] =os.getenv('OPENAI_API_KEY')
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

model =ChatOpenAI(temperature=0,model = 'gpt-4o-mini')

# model.invoke("what is data science")


/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [12]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   
    handle_parsing_errors=True
)

In [13]:
input = ("What is the multiplication of the ratio between stock "
         "prices for 'ABC' and 'XYZ' in January 3rd and the ratio "
         "between the same stock prices in January the 4th?")
print(input)


What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?


In [14]:
result = agent_executor.invoke(input)



> Entering new SQL Agent Executor chain...
Thought: I need to find the stock prices for ABC and XYZ on January 3rd and January 4th. 
Action: sql_db_list_tables
Action Input: stocksThought: I need to find the stock prices for ABC and XYZ on January 3rd and January 4th. 
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Thought: I need to find the stock prices for ABC and XYZ on January 3rd and January 4th. 
Action: sql_db_query_checker
Action Input: SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-03'SELECT price FROM stocks WHERE stock_ticker = 'ABC' AND date = '2023-01-03' 
Thought: I need to find the stock prices for ABC and XYZ on January 3rd and January 4th. 
Action: sql_db_query
Ac

In [15]:
result

{'input': "What is the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th?",
 'output': "The multiplication of the ratio between stock prices for 'ABC' and 'XYZ' in January 3rd and the ratio between the same stock prices in January the 4th is  (232.0/798.0) * (225.0/795.0) = 0.291"}

2 type of agents - conversational agent . these types have memory

In [16]:
from langchain.agents import load_tools

tools = load_tools(
    ["llm-math"],
    llm=llm
)

In [17]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [18]:
conversational_agent.invoke("What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: 10000 * (1 + 0.08)^5
Observation: Answer: 14693.280768000006
Thought:Thought: Do I need to use a tool? No
AI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28. 


> Finished chain.


{'input': "What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?",
 'chat_history': '',
 'output': 'The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.'}

In [19]:
print(conversational_agent.agent.llm_chain.prompt.template)

Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.

Overall, Assistant is a powerful tool that can help with a wide range of tasks 

In [20]:
#now testing the memory
conversational_agent.invoke("If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
# here it remembers the above conversation




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Calculator
Action Input: (15000 * (1 + 0.08)^5) - 14693.28 
Observation: Answer: 7346.641152000009
Thought:Thought: Do I need to use a tool? No
AI: You would have $7,346.64 more. 


> Finished chain.


{'input': 'If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?',
 'chat_history': "Human: What's the result of an investment of $10,000 growing at 8% annually for 5 years with compound interest?\nAI: The result of an investment of $10,000 growing at 8% annually for 5 years with compound interest is $14,693.28.",
 'output': 'You would have $7,346.64 more.'}

3. type docstore agent - will have search and lookup as tools - used to search article and lookup answer

In [21]:
from langchain import Wikipedia
from langchain.agents.react.base import DocstoreExplorer, Tool

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description='search wikipedia'
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description='lookup a term in wikipedia'
    )
]

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `DocstoreExplorer` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0
  warn_deprecated(


In [24]:
docstore_agent = initialize_agent(
    tools,
    llm,
    agent="react-docstore",
    verbose=True,
    max_iterations=3,
    handle_parsing_errors=True
)

In [25]:
docstore_agent.invoke("What were Archimedes' last words?")



> Entering new AgentExecutor chain...
Thought: I need to search Archimedes last words. 
Action: Search[Archimedes last words] 
Observation: A person's last words, their final articulated words stated prior to death or as death approaches, are often recorded because of the decedent's fame, but sometimes because of interest in the statement itself. (People dying of illness are frequently inarticulate at the end, and in such cases their actual last utterances may not be recorded or considered very important.) Last words may be recorded accurately, or, for a variety of reasons, may not. Reasons can include simple error or deliberate intent. Even if reported wrongly, putative last words can constitute an important part of the perceived historical records or demonstration of cultural attitudes toward death at the time.
Charles Darwin, for example, was reported to have disavowed his theory of evolution in favor of traditional religious faith at his death. This widely disseminated report ser

{'input': "What were Archimedes' last words?",
 'output': 'Agent stopped due to iteration limit or time limit.'}

self ask with search - the agent will ask questions to get and use search to get answers and this keep on going to find a best answer

In [ ]:
from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool

search = SerpAPIWrapper(serpapi_api_key=os.getenv('serp_api_key'))
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description='google search'
    )
]

In [ ]:
self_ask_with_search = initialize_agent(tools, llm, agent="self-ask-with-search", verbose=True)

In [ ]:
self_ask_with_search.invoke("what was jackie chan doing in the 1950")

type of llm memory 


how to inspect the langchain functions

In [26]:
import inspect

from getpass import getpass
from langchain import OpenAI
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.conversation.memory import (ConversationBufferMemory,
                                                  ConversationSummaryMemory,
                                                  ConversationBufferWindowMemory,
                                                  ConversationKGMemory)
from langchain.callbacks import get_openai_callback
import tiktoken

In [27]:
conversation = ConversationChain(
    llm=llm,
)

In [28]:
print(conversation.prompt.template)


The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [31]:
print(inspect.getsource(conversation._call), inspect.getsource(conversation.apply))

    def _call(
        self,
        inputs: Dict[str, Any],
        run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, str]:
        response = self.generate([inputs], run_manager=run_manager)
        return self.create_outputs(response)[0]
     def apply(
        self, input_list: List[Dict[str, Any]], callbacks: Callbacks = None
    ) -> List[Dict[str, str]]:
        """Utilize the LLM generate method for speed gains."""
        callback_manager = CallbackManager.configure(
            callbacks, self.callbacks, self.verbose
        )
        run_manager = callback_manager.on_chain_start(
            dumpd(self),
            {"input_list": input_list},
        )
        try:
            response = self.generate(input_list, run_manager=run_manager)
        except BaseException as e:
            run_manager.on_chain_error(e)
            raise e
        outputs = self.create_outputs(response)
        run_manager.on_chain_end({"outputs": outputs})
        retur

types of memory 

In [32]:
#here the entire conversation is the given back to the llm - this is token high as all the
#the conversation is given to the llm 
conversation_buf = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [33]:
conversation_buf("Good morning AI!")

/home/varun/anaconda3/envs/genai/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'input': 'Good morning AI!',
 'history': '',
 'response': "Good morning!  It's lovely to hear from you.  The sun is shining brightly here in the digital world. Is it a beautiful day where you are? ☀️  \n\n"}

In [34]:

    conversation_buf("My interest here is to explore the potential of integrating Large Language Models with external knowledge")


{'input': 'My interest here is to explore the potential of integrating Large Language Models with external knowledge',
 'history': "Human: Good morning AI!\nAI: Good morning!  It's lovely to hear from you.  The sun is shining brightly here in the digital world. Is it a beautiful day where you are? ☀️  \n\n",
 'response': "That's a fascinating topic! I can definitely see the potential there. Integrating Large Language Models with external knowledge would be like giving me superpowered abilities. \n\nImagine being able to access and process real-time information, up-to-date facts, and specialized databases directly within my responses. I could provide much more accurate, relevant, and insightful answers to your questions. \n\nFor example, instead of just telling you that the capital of France is Paris, I could tell you the population of Paris, its historical significance, and even some fun facts about famous landmarks there. \n\nWhat specific aspects of this integration are you most inte

conversation summary buffer memory - here the previous conversation is summarized and given to the llm 

In [35]:
conversation_sum = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm)
)

conversation buffer window memory - here only the latest conversations is given back and the first ones are disregarded


In [36]:
conversation_bufw = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=1)
)